Tiering of BloodHound data

Import libs

In [6]:
from neo4j import GraphDatabase
import pathlib

Create connection

In [7]:
class Neo4jConnection:
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
                
        return response
conn = Neo4jConnection(uri="bolt://localhost:7687", user="neo4j", pwd="vovse")
relpath = pathlib.Path().absolute().as_uri()

In [8]:
query = '''
CALL apoc.load.json('%s/../jsondata/bloodhound/tiering-bloodhound.json') YIELD value
UNWIND value.groups AS g
MERGE (tier:Tier {level:g.tier})
WITH g, tier
MATCH (group:Group) WHERE group.objectid ENDS WITH g.RID
MATCH (principal)-[:MemberOf*1..]->(group) WHERE (principal:User) OR (principal:Computer)
MERGE (principal)-[tierrel:InTier]->(tier)

// Delete any other tier relations for principal
WITH principal, tierrel
MATCH (principal)-[old:InTier]->() WHERE old <> tierrel
DELETE old
''' % (relpath)
out = conn.query(query)